In [ ]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import os

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import tree

import matplotlib.pyplot as plt
from itertools import product
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Regresión Logística

Algoritmo de CLASIFICACIÓN, que tradicionalmente es binario

In [ ]:
df = pd.read_csv(os.getcwd() + "\\data\\2008_small.csv",nrows=100000)
df = df[["AirTime","Distance","TaxiOut","ArrDelay","DepDelay"]].dropna()

X = df[df.columns[df.columns != "ArrDelay"]]
filtro = df["ArrDelay"] > 10 
df["ArrDelay"][filtro] = "Delayed"
df["ArrDelay"][filtro== False] = "Not Delayed"
Y = df["ArrDelay"]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=2)

In [ ]:
clf = LogisticRegression(penalty = "none").fit(X_train, y_train)
predicciones = clf.predict(X_test)

In [ ]:
confusion = confusion_matrix(y_test,predicciones)
ax = sns.heatmap( confusion, annot=True,fmt='.6g',
            xticklabels= clf.classes_, 
            yticklabels=clf.classes_)

# bottom, top = ax.get_ylim()
# ax.set_ylim(bottom + 0.5, top - 0.5)

In [ ]:
y_test.value_counts() / len(y_test)

In [ ]:
np.mean(predicciones == y_test)

In [ ]:
f1_score(y_test, predicciones, average='macro')

In [ ]:
clf.classes_

In [ ]:
predicciones

In [ ]:
clf.predict_proba(X_test).round(4)

In [ ]:
prob_delayed = clf.predict_proba(X_test)[:,0]

delayedseguras = prob_delayed > 0.6
notdelayedseguras = prob_delayed < 0.4
torevise = (prob_delayed >= 0.4) & (prob_delayed <= 0.6)

In [ ]:
X_test[delayedseguras]

# Árbol de Regresión

In [ ]:
df = pd.read_csv(os.getcwd() + "\\data\\2008_small.csv",index_col = 0, nrows = 200000)

In [ ]:
df = df.dropna(subset = ["ArrDelay"])
df = df[df.AirTime > 0]

X = df[['AirTime','Distance','TaxiIn','TaxiOut', 'DepDelay']]
Y = df[["ArrDelay"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=2)

arbol = tree.DecisionTreeRegressor()
arbol = arbol.fit(X_train, y_train)

prediccionesINTERNA = arbol.predict(X_train)
print("R2 INTERNA: ",r2_score(y_train, prediccionesINTERNA))

prediccionesEXTERNA = arbol.predict(X_test)
print("R2 EXTERNA: ",r2_score(y_test, prediccionesEXTERNA))

Ajuste de parámetros (ensayo y error)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=2)

In [ ]:
arbol = tree.DecisionTreeRegressor(max_depth=10)
arbol = arbol.fit(X_train, y_train)

prediccionesINTERNA = arbol.predict(X_train)
print("R2 INTERNA: ",r2_score(y_train, prediccionesINTERNA))

prediccionesEXTERNA = arbol.predict(X_test)
print("R2 EXTERNA: ",r2_score(y_test, prediccionesEXTERNA))

### Ensayo de parámetros (más automatizado)

Estructura de guardado de datos (basada en una función de R, llamada expand.grid)

In [ ]:
from itertools import product

# Esta función no hace falta entenderla en detalle :)
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

parametros = {'semillas': range(0,15), # Vamos a hacer 10 conjuntos distintos de train/test
              'profundidades': range(3,25,1), #Vamos a probar profundidades 1, 4, 7..
              'splitter': ["best", "random"], #Y vamos a seleccionar la variable de cada partición de manera óptima / aleatoria
              'R2': [np.nan]}

dfparam = expand_grid(parametros) # Crea un dataframe con todas las combinaciones de parámetros
dfparam.head(10)

In [ ]:
for semilla in tqdm(parametros["semillas"]):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=semilla)
    for prof in parametros["profundidades"]:
        for split in parametros["splitter"]:
  
            arbol = tree.DecisionTreeRegressor(max_depth=prof, splitter=split)
            arbol = arbol.fit(X_train, y_train)

            prediccionesEXTERNA = arbol.predict(X_test)
            dfparam.loc[(dfparam.semillas == semilla) & (dfparam.profundidades == prof) & (dfparam.splitter == split),"R2"] = r2_score(y_test, prediccionesEXTERNA)


In [ ]:
for i in tqdm(range(dfparam.shape[0])):
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, 
                                                        random_state=dfparam.loc[i,"semillas"])
    
    arbol = tree.DecisionTreeRegressor(max_depth=dfparam.loc[i,"profundidades"], 
                                       splitter=dfparam.loc[i,"splitter"])
    arbol = arbol.fit(X_train, y_train)

    prediccionesEXTERNA = arbol.predict(X_test)
    dfparam.loc[i,"R2"] = r2_score(y_test, prediccionesEXTERNA)


In [ ]:
dfparam.sort_values("R2",ascending=False)

In [ ]:
dfparam.groupby("profundidades").mean()["R2"]

In [ ]:
dfparam.groupby("splitter").mean()["R2"].sort_values(ascending=False)

In [ ]:
dfparam.groupby("semillas").mean()["R2"].sort_values(ascending=False)

In [ ]:
dfparam.groupby(["profundidades","splitter"]).mean()["R2"]

In [ ]:
dfparam.sort_values("R2",ascending=False).head(20)

In [ ]:
dfparam.sort_values("R2",ascending=False).tail(20)

# Ejercicios

In [ ]:
sns.load_dataset("mpg")

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from itertools import product
df = pd.read_excel("data\cars.xlsx", index_col=0).dropna() #Carga de datos (limpios)
df = df.reset_index() #Evitamos que aparezcan índices sin datos en el dataframe

# Importación de una función personalizada que permite conseguir un dataframe con todas las combinaciones de parámetros (requiere un diccionario)
def expand_grid(dictionary):
    return pd.DataFrame([row for row in product(*dictionary.values())], columns=dictionary.keys())

#Dummies
df["cylinders_cat"] = df["cylinders"].astype("str") # Conversión a string de las variables numéricas que queremos convertir a dummies
df["model_year_cat"] = df["model_year"].astype("str")
df["interaccion"] =  df["model_year_cat"] + " " + df["cylinders_cat"] # Interacción entre variables categóricas
dummies = pd.get_dummies(df[["origin","interaccion"]]) # Conversión a dummies de las variables categóricas

Y = df[["mpg"]] # Selección de la variable respuesta
X = df[["displacement","horsepower","weight","acceleration","model_year","cylinders"]] #Selección de las variables numéricas explicativas

# Escalado
transformados = preprocessing.scale(X) # Escalado de datos (no se guardan)
#X = pd.DataFrame(transformados, columns = X.columns) # Descomentar si queremos escalar datos numéricos

# Polinomios
poly = PolynomialFeatures(interaction_only=True) # Creo las interacciones entre variables numéricas
X_poly = poly.fit_transform(X) # Creo la variable con las interacciones
X = pd.DataFrame(X_poly, columns = poly.get_feature_names(X.columns)) #Sobreescribo X
X = pd.concat([X,dummies],axis = 1) # Descomentar si queremos añadir dummies

#Creamos el dataframe con las combinaciones de parámetros y semillas que queremos probar
parametros = {'semillas': range(0,20),'profundidades': range(2,30,1),'splitter': ["best", "random"],'R2': [np.nan]}
dfparam = expand_grid(parametros) 

# Entrenamos y evaluamos todas las combinaciones! (bucle por filas del dataframe de parámetros)
for i in tqdm(range(dfparam.shape[0])):
    #Creamos el conjunto train test con la semilla correspondiente
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, 
                                                        random_state=dfparam.loc[i,"semillas"])
    
    # Creamos el árbol con los parámetros que le corresponden
    arbol = tree.DecisionTreeRegressor(max_depth=dfparam.loc[i,"profundidades"], 
                                       splitter=dfparam.loc[i,"splitter"],random_state=1)
    arbol = arbol.fit(X_train, y_train) # Ajustamos el árbol con los datos

    prediccionesEXTERNA = arbol.predict(X_test) # Hacemos las predicciones con los datos que el modelo aún no ha visto
    dfparam.loc[i,"R2"] = r2_score(y_test, prediccionesEXTERNA) # Añadimos al dataframe los resultados del R2 externo

# Mostramos las mejores combinaciones de parámetros y semilla
resultados = dfparam.groupby(["profundidades","splitter"]).mean().sort_values("R2",ascending = False)["R2"]
resultados

In [ ]:
resultados.index[0]

In [ ]:
resultados.index[0]

arbol = tree.DecisionTreeRegressor(max_depth=resultados.index[0][0], 
                                   splitter=resultados.index[0][1],
                                   random_state=1)
arbol = arbol.fit(X, Y)

1. Estudia el dataframe, se llama mpg, puedes buscar qué significan exactamente las variables, y hacer una descripción (numérica y gráfica) de las que te parezcan más relevantes

2. Ajusta un modelo predictivo sencillo para el el consumo del coche (mpg). Evalúa su potencial predictivo tanto interna como externamente

3. Ajusta algunos de los parámetros que conocemos (o alguno nuevo) para intentar mejorar las predicciones

4. Ajusta los parámetros que creas convenientes (o explora algunos nuevos), intenta replicar una estructura de tipo bucle para hacerlo, mostrando los mejores resultados que hayas encontrado

### Clasificación 

1. Ajusta un modelo predictivo para el origen de los coches, o categoriza una variable como el consumo o el peso de los coches

2. Evalúa el modelo predictivo de varias formas

3. Ajusta los parámetros que creas convenientes, intenta replicar una estructura de tipo bucle para hacerlo, mostrando los mejores resultados que hayas encontrado

4. Compara distintos modelos de clasificación, cuál funciona mejor?

### Clustering

1. Intenta agrupar los coches usando el algoritmo DBSCAN

2. Representa gráficamente los resultados, qué cantidad de grupos te parecen razonables?

3. Juega con el parámetro eps a ver qué diferencias se observan

4. Compara con los demás algoritmos de clústering que hemos visto!

5. Hay alguna correspondencia entre los grupos generados y la variable "origin"?